In [1]:
import yaml
import copy
import os
import numpy as np

In [2]:
config_base_path="./configs"

In [3]:
base_config = {
    "batch_size": 128,
    "dataset_path": "cfpark00/toy-multistep-nn_5-na_5-nab_5-seed_0",
    "dataset_split": "train",
    "instruct_mode": False,
    "model_name": "cfpark00/toy-multistep-nn_5-na_5",
    "model_parallel": False,
    "save_path": "./data/sft/toy-multistep-nn_5-na_5-nab_5-seed_0",
    "train_max_length": 64,
    "training_arguments": {
        "bf16": False,
        "gradient_accumulation_steps": 1,
        "learning_rate": 0.001,
        "logging_steps": 12,
        "lr_scheduler_type": "linear",
        "warmup_ratio": 0.05,
        "num_train_epochs": 1,
        "save_only_model": True,
        "save_steps": 0.2,
        "save_strategy": "no"
    },
    'wandb': {
        'project_name': 'toy-multistep-reasoning',
        'run_name': 'toy-multistep-reasoning-example-nn_5-na_5-nab_5-seed_0'
    }
}

In [23]:
config_paths=[]
for n_nodes in [5,10,20]:
    for n_actions in [5,10,20]:
        for n_ablate in ["0.1","0.2","0.3"]:
            for seed in [0,1,2]:
                n_edges=n_nodes*n_actions
                if type(n_ablate) == str:
                    n_ablate = int(float(n_ablate)*n_edges)
                config= copy.deepcopy(base_config)
                config["dataset_path"] = f"cfpark00/toy-multistep-nn_{n_nodes}-na_{n_actions}-nab_{n_ablate}-seed_{seed}"
                config["model_name"] = f"cfpark00/toy-multistep-nn_{n_nodes}-na_{n_actions}"
                config["save_path"] = f"./data/sft/toy-multistep-nn_{n_nodes}-na_{n_actions}-nab_{n_ablate}-seed_{seed}"
                config["wandb"]["run_name"] = f"toy-multistep-reasoning-example-nn_{n_nodes}-na_{n_actions}-nab_{n_ablate}-seed_{seed}"

                config_path= os.path.join(config_base_path, f"nn_{n_nodes}-na_{n_actions}-nab_{n_ablate}",f"seed_{seed}.yaml")
                os.makedirs(os.path.dirname(config_path), exist_ok=True)
                with open(config_path, 'w') as f:
                    yaml.dump(config, f)
                config_paths.append(config_path)

In [24]:
script_path = "./scripts/train.sh"
with open(script_path, 'w') as f:
    f.write("#!/bin/bash\n")
    for config_path in config_paths:
        f.write(f"echo {config_path}\n")
        f.write(f"accelerate launch /n/home12/cfpark00/ML/tools/run_sft_accelerate.py {config_path} --overwrite\n")
        f.write("echo \"\"\n")

In [4]:
eval_script_path = "./scripts/eval.sh"
with open(eval_script_path, 'w') as f:
    f.write("#!/bin/bash\n")
    for n_nodes in [5,10,20]:
        for n_actions in [5,10,20]:
            for n_ablate in ["0.1","0.2","0.3"]:
                for seed in [0,1,2]:
                    n_edges=n_nodes*n_actions
                    if type(n_ablate) == str:
                        n_ablate = int(float(n_ablate)*n_edges)
                    f.write(f"echo nn_{n_nodes}-na_{n_actions}-nab_{n_ablate}-seed_{seed}\n")
                    f.write(f"python3 eval.py --n_n {n_nodes} --n_a {n_actions} --n_ab {n_ablate} --seed {seed} --n_problems 4096 \n")